This project requires AWS account!

You can then create a jupyter-notebook instance (the free one works well). After that you can upload this notebook and run it.


# Download the Data

In [1]:
!pip3 install tqdm
!pip3 install pillow --upgrade

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 78 kB 3.0 MB/s eta 0:00:01
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 3.1 MB 5.9 MB/s eta 0:00:01


In [2]:
%matplotlib inline

import os
import tarfile
import urllib
import shutil
import json
import random
import numpy as np
import boto3
import sagemaker

from tqdm import tqdm
from sagemaker.amazon.amazon_estimator import get_image_uri
from matplotlib import pyplot as plt
from xml.etree import ElementTree as ET
from PIL import Image, ImageDraw, ImageFont

urls_list = ['http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz',
        'http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz']

def download_and_extract(data_dir, download_dir, urls):
    for url in urls:
        target_file = url.split('/')[-1]
        if target_file not in os.listdir(download_dir):
            print('Downloading', url)
            urllib.request.urlretrieve(url, os.path.join(download_dir, target_file))
            tf = tarfile.open(url.split('/')[-1])
            tf.extractall(data_dir)
        else:
            print('Already downloaded', url)

if not os.path.isdir('data'):
    os.mkdir('data')

download_and_extract('data', '.', urls_list)

ModuleNotFoundError: No module named 'boto3'

# Extracting Annotations from XML Format

In [ ]:
xml_dir = 'data/annotations/xmls/'
xml_files = [os.path.join(xml_dir, x) for x in os.listdir(xml_dir) if x[-3:] == 'xml']
xml_files[0]

In [ ]:
classes = ['cat', 'dog']
categories = [
    {
        'class_id': 0,
        'name': 'cat'
    },
    {
        'class_id': 1,
        'name': 'dog'
    }
]

There are a few ways to provide data to sagemaker algorithms. 
- One of the ways is to create a record file in a format which, which the mxnet framework expects, because the sagemaker algorithms are actually built on top of mxnet.
- A more generic and more user friendly way is providing data as JSON files to these algorithms. But the trick is that those Jason Files need to follow a very strict pattern and adhere to a strict pattern so that the record files can be created from these Jason Files before the algorithm actually starts to train.


So at a pre processing step, we will actually generate the required data set files by reading these Jason annotation
files. So we cannot use the X similar annotations as is. And that is the reason why we need to ah, convert the XML
annotations to Jason annotations and the structure is pretty straightforward.

What we will do is we will take a look at an XML file and then we'll find the file name from XML, and the key file in our Json annotations will be set to the categories defined before.


In [ ]:
def extract_annotation(xml_file_path):
    
    tree = ET.parse(xml_file_path)
    root = tree.getroot()
    annotation = {}
    
    annotation['file'] = root.find('filename').text
    annotation['categories'] = categories
    
    size = root.find('size')
    
    annotation['image_size'] = [{
        'width': int(size.find('width').text),
        'height': int(size.find('height').text),
        'depth': int(size.find('depth').text)
    }]
    
    annotation['annotations'] = []
    
    for item in root.iter('object'):
        class_id = classes.index(item.find('name').text)
        ymin, xmin, ymax, xmax = None, None, None, None
        
        for box in item.findall('bndbox'):
            xmin = int(box.find("xmin").text)
            ymin = int(box.find("ymin").text)
            xmax = int(box.find("xmax").text)
            ymax = int(box.find("ymax").text)
        
            if all([xmin, ymin, xmax, ymax]) is not None:
                 annotation['annotations'].append({
                     'class_id': class_id,
                     'left': xmin,
                     'top': ymin,
                     'width': xmax - xmin,
                     'height': ymax - ymin
                 })
    return annotation

In [ ]:
extract_annotation(xml_files[0])

# Visualize Data

In [ ]:
def plot_example(plt, annot, image_file_path, rows, cols, index):
    img = Image.open(image_file_path)
    draw = ImageDraw.Draw(img)
    font = ImageFont.truetype('/usr/share/fonts/dejavu/DejaVuSerif-Bold.ttf', 20)
    
    for a in annot['annotations']:
        box = [
            int(a['left']), int(a['top']),
            int(a['left']) + int(a['width']),
            int(a['top']) + int(a['height'])
        ]
        draw.rectangle(box, outline='yellow', width=4)
        draw.text((box[0], box[1]), classes[int(a['class_id'])], font=font)
    plt.subplot(rows, cols, index + 1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img)
    return plt

def show_random_annotations(plt):
    plt.figure(figsize=(16, 16))
    for i in range(0, 9):
        index = random.randint(0, len(xml_files) - 1)
        
        annot = extract_annotation(xml_files[index])
        image_file_path = os.path.join('data/images/', annot['file'])

        plt = plot_example(plt, annot, image_file_path, 3, 3, i)
    plt.show()

In [ ]:
show_random_annotations(plt)

# SageMaker Setup

First we create the role, which bis needed when we create a sagemaker estimator, we will also need
a bucket on.

In [ ]:
role = sagemaker.get_execution_role()
bucket_name = 'petsdata'

training_image = get_image_uri(boto3.Session().region_name(), 'object-detection', repo_version = 'latest')

print(training_image)

In [ ]:
folders = ['train', 'train_annotation', 'validation', 'validation_annotation']

for folder in folders:
    if os.path.isdir(folder):
        shutil.rmtree(folder)
    os.mkdir(folder)

# Preparing Data for SageMaker

In [ ]:
print('Total examples:', len(xml_files))

In [ ]:
for xml_file in tqdm(xml_files):
    target_set = 'train' if random.randint(0, 99) < 75 else 'validation'
    annot = extract_annotation(xml_file)
    image_file_path = os.path.join('data/images/', annot['file'])
    image_target_path = os.path.join(target_set, annot['file'])
    shutil.copy(image_file_path, image_target_path)
    json_file_path = os.path.join(target_set + '_annotation', annot['file'][:-3] + 'json')
    with open(json_file_path, 'w') as f:
        json.dump(annot, f)

In [ ]:
train_images = os.listdir('train')
train_annots = os.listdir('train_annotation')

In [ ]:
print(len(train_annots), len(train_images))

In [ ]:
# This is a check to check if all the file names to matchup.
for image in train_images:
    key = image.split('.')[0]
    json_file = key + '.json'
    if json_file not in train_annots:
        print('Not found', json_file)

# Uploading Data to S3
This process takes a bit of time, up to 10 min

In [ ]:
# First we start a session
sees = sagemaker.Session()

# We upload the images to S3 to train it
print('Uploading data ..') # This is check point
s3_train_path = sess.upload_data(path='train', bucket= bucket_name, key_perfix= 'train')
print('Data is uploaded ..')
s3_validation_path = sess.upload_data(path='validation', bucket= bucket_name, key_perfix= 'validation')

# Upload the annotation data
print('Uploading annotation data ..')
s3_train_annotations = sess.upload_data(path='train_annotations', bucket= bucket_name, 
                                        key_perfix= 'train_annotations')
print('Annotation data is uploaded ..')
s3_validation_annotations = sess.upload_data(path='validation_annotations', bucket= bucket_name, 
                                             key_perfix= 'validation_annotations')


In [ ]:
s3_validation_annotations # This will return where the validation annotation is located

# SageMaker Estimator
Now let's create a safe to make an estimator, which is
basically a high level API which will handle the training job
for us.

- So we need to specify the training image and set an execution role.
- For training, we use an instance type of ml dot Petri 0.0.2 x Large, this instance has 16 GB of GPU memory. We will use only one instance.

- Next set the training volume size 100 TB
- Set a time limit for this training job 36,000 seconds.
- When you use the decent annotations like we have, you want to use the file mode.
- The output path is the path where you will store the train model artifact and let's store it in the in a in a new full record output in pets data bucket 
- finally, we will specify sagemaker session.

In [ ]:
model = sagemaker.estimator.Estimator(
    training_image,
    role = role,
    train_instance_type ='ml.p3.2xlarge',
    train_instance_count = 1,
    train_volume_size = 100,
    train_max_rum = 3600,
    input_mode = 'File',
    output_path = 's3://petsdata/output',
    sagemaker_session = sess
)

# Hyperparameters
Which are relevant with respect to the object
detection SSD algorithm that we are using.

In [ ]:
model.set_hyperparameters(
    base_network = 'resnet-50',
    num_classes = 2,
    use_pretrained_model = 1,
    mini_batch_size = 16,
    epochs = 15,
    learning_rate = 0.001,
    optimizer = 'sgd',
    lr_scheduler_step = '10',
    le_scheduler_factor = 0.1,
    momentum = 0.9,
    weight_decay = 0.0005,
    overlap_threshold = 0.45,
    image_shape = 512,
    num_training_samples = len(train_annots)
)

# Data Channels

In [ ]:
from sagemaker.session import s3_input
train_data = s3_input(s3_train_path,
                       distribution = 'FullyReplicated',
                       content_type = 'application/x-image',
                       s3_data_type = 'S3Prefix')

validation_data = s3_input(s3_validation_path,
                       distribution = 'FullyReplicated',
                       content_type = 'application/x-image',
                       s3_data_type = 'S3Prefix')

train_annotation_data = s3_input(s3_train_annotations,
                       distribution = 'FullyReplicated',
                       content_type = 'application/x-image',
                       s3_data_type = 'S3Prefix')

validation_annotations_data = s3_input(s3_validation_annotations,
                       distribution = 'FullyReplicated',
                       content_type = 'application/x-image',
                       s3_data_type = 'S3Prefix')

In [ ]:
data_channels = {
    'train': train_Data,
    'validation': validation_data,
    'train_annotation': train_annotation_data,
    'validation_annotation': validation_annotations_data
}

# Model Training

In [ ]:
model.fit(inputs = data_channel, logs = True)

# Deploy Model
A sagemaker is going to spin
up an instance of this type, and then it will copy the model
artifact to this instance and ready for influence
and basically start serving it.

In [ ]:
deployed_model = model.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')
print('\nModeldeployed!')

# Predictions

In [ ]:
image_dir = 'validation'
images = [x for x in os.listdir(image_dir) if x[-3:] == 'jpg']
print(len(images))

In [ ]:
deployed_model.content_type = 'image/jpeg'

In [ ]:
index = 0

image_path = os.path.join(image_dir, images[index])
# image_path = 'dog_cat.jfif'

with open(image_path, 'rb') as f:
    b = bytearray(f.read())

results = deployed_model.predict(b)
results = json.loads(results)

preds = results['prediction']

In [ ]:
preds[0]

In [ ]:
img = Image.open(image_path)
draw = ImageDraw.Draw(img)
font = ImageFont.truetype('/usr/share/fonts/dejavu/DejaVuSerif-Bold.ttf', 30)
w, h =img.size

for pred in preds:
    # extract info out of the predictions
    class_id, score, xmin, ymin, xmax, ymax = pred
    
    if score > 0.7:
        box = [w*xmin, h*ymin, w*xmax, h*ymax]

        draw.rectangle(box, outline='yellow', width=4)
        draw.text((box[0], box[1]), classes[int(class_id)], font=font, fill='#000000')
    else:
        break

plt.xticks([])
plt.yticks([])
plt.imshow(img)
plt.show()

Don't forget!! You need to delete endpoint or else you will continue to accrue cost!

In [ ]:
sagemaker.Session().delete_endpoint(deployed_model.endpoint)